# Good random segments, in Smith

There is a lot of data in the Smith set. Less than half has oscillations, see `smith_analysis_oscillation_log.md`. To find segments of data with oscillations, I turn to FOOOF.

FOOOF needs python3. So I put this path of project into an independent notebook. 

In [1]:
# -
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row, gridplot
from bokeh.models import Range1d
from bokeh.io import export_png
output_notebook()

# -
from fooof import FOOOF

import csv
import os
import h5py
import numpy as np

from tqdm import tqdm
from glob import glob
from pprint import pprint


from scipy.io import loadmat
from scipy.signal import medfilt
from scipy.signal import resample
from scipy.signal import welch, gaussian
from scipy.signal import butter, lfilter
from scipy.signal import hilbert

# --
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')

    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)

    return y

# -
def plot_grid_psd(x, m, fs, show_plot=False):
    freqs, psd = create_psd(x[m], fs)
    p = figure(plot_width=150, plot_height=150)
    p.line(freqs, np.log10(psd), color="black", alpha=1)
    p.xaxis.axis_label = 'Freq (Hz)'
    p.yaxis.axis_label = 'Log power (AU)'
    p.x_range = Range1d(1, 50)
    p.y_range = Range1d(-3, 5)
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.toolbar.logo = None
    p.toolbar_location = None
    
    if show_plot:
        show(p)
    else:
        return p

# -
def random_fooof(X,
                 num_windows,
                 window_length,
                 good_channels,
                 fs=1000,
                 alpha_range=(8, 16),
                 fit_range=(3, 40),
                 i_min=10000,
                 max_iterations=25000,
                 save_results=False):
    """FOOOF random channels and windows of X."""
    
    # Init
    fm = FOOOF(peak_width_limits=[2.0, 12.0])

    # Est baseline std for all channels
    std_ref = X.std()
    mean_alpha = np.mean(alpha_range)

    # -
    m = 0  # Window count
    k = 0  # Iter count
    n_samples = X.shape[0] 
    
    results = []
    while m < num_windows:
        # Update overall iter count
        k += 1
        if k > max_iterations:
            break

        # Sample random channel from the good
        np.random.shuffle(good_channels)
        c = good_channels[0]
    
        x = X[:, c]
        

        if i_min > n_samples:
            raise ValueError("i_min must be less than {}".format(i_max))
            
        # -
        # Find a good window
        stop_idx_search = True
        while stop_idx_search:
            # Update overall iter count
            k += 1
            if k > max_iterations:
                break
            
            # Generate random i:j 
            i = np.random.randint(i_min, n_samples - window_length, 1)
            j = i + window_length
                  
            i = int(i)
            j = int(j)
            
            # Basic QC pass?
            if x[i:j].std() > (3 * std_ref):
                continue

            if j > n_samples:
                raise ValueError(
                    "i ({}) exceeded n_samples {}".format(i, n_samples))
            
            stop_idx_search = False
            
        # FOOOF x in the window
        freqs, psd = create_psd(x[i:j], fs)
        fm.fit(freqs, psd, fit_range)

        # -
        # Repack peak_params_ into seperate values
        centers = []
        powers = []
        bws = []
        for (center, amp, bw) in fm.peak_params_:
            centers.append(center)
            powers.append(amp)
            bws.append(bw)
        
        centers = np.asarray(centers)
        powers = np.asarray(powers)
        bws = np.asarray(bws)
        
        # Found any peaks?
        if centers.size == 0:
            continue
        
        # -
        # Find closest to mean alpha
        idx = (np.abs(centers - mean_alpha)).argmin()
        closest_peak = centers[idx]

        # It is in range?
        if (closest_peak >= alpha_range[0]) and (
                closest_peak <= alpha_range[1]):

            row = (m, c, i, j, closest_peak, powers[idx], bws[idx])
            results.append(row)

            m += 1

    return results

def save_fooof_results(name, results):
    header = ("m", "c", "i", "j", "center", "power", "bw")

    with open(name, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(header)
        
        for row in results:
            writer.writerow(row)

            
def load_foof_results(name):
    results = []
    with open(name, 'r') as fi:
        reader = csv.reader(fi, delimiter=",")
        for i, row in enumerate(reader):
            results.append(row)

    header = results.pop()
    
    return header, results


def create_psd(lfp, inrate, outrate=1000):
    """Calculate PSD from LFP/EEG data."""
    lfp = np.array(lfp)

    if inrate != outrate:
        lfp = resample(lfp, int(lfp.shape[0] * outrate / inrate))

    # Calculate PSD
    return welch(
        lfp,
        fs=outrate,
        window='hanning',
        nperseg=outrate,
        noverlap=outrate / 2.0,
        nfft=None,
        detrend='linear',
        return_onesided=True,
        scaling='density')

def create_times(t, dt):
    n_steps = int(t * (1.0 / dt))
    times = np.linspace(0, t, n_steps)

    return times

Loading BokehJS ...

# Data path

In [2]:
DATA_PATH = "/Users/type/Data/Smith/data"

# Good files

Found by manual plotting. See `smith_analysis_oscillation_log.md`.

In [3]:
files = ["Bo130408_s6ae_fixblank_active_0001_converted_ns2.mat",
"Bo130408_s6ae_fixblank_active_0002_converted_ns2.mat",
"Bo130409_s7ae_fixblank_active_0003_converted_ns2.mat",
"Wi130116_s51ae_fixblank_active_0001_converted_ns2.mat",
"Bo130404_s4ae_fixblank_active_0002_converted_ns2.mat",
"Bo130405_s5ae_fixblank_active_0001_converted_ns2.mat",
"Bo130405_s5ae_fixblank_active_0002_converted_ns2.mat",
"Bo130405_s5ae_fixblank_active_0003_converted_ns2.mat",
"Bo130418_s12ae_fixblank_active_0002_converted_ns2.mat",
"Wi121219_s43ae_fixblank_active_0001_converted_ns2.mat",
"Wi121219_s43ae_fixblank_active_0002_converted_ns2.mat",
"Wi130129_s55ae_fixblank_active_0001_converted_ns2.mat",
"Wi130129_s55ae_fixblank_active_0002_converted_ns2.mat",
"Wi130205_s58ae_fixblank_active_0001_converted_ns2.mat",
"Wi130205_s58ae_fixblank_active_0002_converted_ns2.mat",
"Wi130207_s59ae_fixblank_active_0001_converted_ns2.mat",
"Wi130207_s59ae_fixblank_active_0002_converted_ns2.mat",
"Wi130207_s59ae_fixblank_active_0003_converted_ns2.mat",
"Wi130208_s60ae_fixblank_active_0001_converted_ns2.mat",
"Wi130211_s61ae_fixblank_active_0001_converted_ns2.mat",
"Wi130212_s62ae_fixblank_active_0001_converted_ns2.mat"]

# Choose an example file

In [4]:
i = 0
print("Running {}".format(files[i]))

Running Bo130408_s6ae_fixblank_active_0001_converted_ns2.mat


# Load the handle

In [5]:
fi = h5py.File(os.path.join(DATA_PATH, files[i]))
pprint(list(fi.keys()))

['#refs#',
 'Fs',
 'chanunit',
 'data',
 'label',
 'nChans',
 'nSamples',
 'scale',
 'timeStamps']


# Create time

In [6]:
fs = float(fi['Fs'].value)
n_samples = float(fi['nSamples'].value) 
T = n_samples / fs
print("Experiment time T {}".format(T))

times = create_times(T, 1/fs)
print("Times: {}".format(times[:20]))

Experiment time T 3486.306
Times: [ 0.          0.001       0.002       0.003       0.004       0.005       0.006
  0.007       0.008       0.009       0.01        0.011       0.012       0.013
  0.014       0.015       0.016       0.017       0.01800001  0.01900001]


# Get data

and print stats.

In [7]:
X = fi['data'].value
print("Data shape: {}".format(X.shape))

Data shape: (3486306, 104)


# Example random FOOOF!

In [8]:
results = random_fooof(X, 10, 10*fs, list(range(0, 95)), fs=fs)

/Users/type/anaconda/envs/py3/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [9]:
pprint(results[:3])
save_fooof_results("test.csv", results)

[(0,
  82,
  996564,
  1006564,
  13.120495413143001,
  0.4266003319647278,
  3.8430732080439958),
 (1,
  62,
  1189038,
  1199038,
  9.3634144391113221,
  0.27757741888425569,
  3.5632169095804933),
 (2,
  54,
  2120091,
  2130091,
  10.907014260519945,
  0.36302118818209239,
  6.0054855545238377)]


- Results seem sane

# FOOOF all the good files

- 6000 samples / file (or 600 per electrode on average)

### LFP

In [10]:
sample_percent = 0.1
channels = list(range(0, 95))

for fi in tqdm(files):
    # Load the handle
    hdf = h5py.File(os.path.join(DATA_PATH, fi))
    
    # Extract time information
    fs = float(hdf['Fs'].value)
    n_samples = float(hdf['nSamples'].value) 
    T = n_samples / fs
    
    # Number of samples to pull, as a percent of T
    n_window = int(n_samples * sample_percent)
    
    # Window length (in samples)
    l = 10*fs
    
    # Get the data
    X = hdf['data'].value

    # Extract name, drop extension
    fi_name = os.path.splitext(fi)[0]
    
    # Do random FOOOF
    results = random_fooof(X, n_window, l, channels, fs=fs)
    
    # Save the result
    save_fooof_results(
        "{}_segments.csv".format(os.path.join(DATA_PATH, fi_name)), 
        results)

100%|██████████| 21/21 [2:00:42<00:00, 344.90s/it]


# Examine random random segments.

Find all the segment files

In [ ]:
# Find all the segments files (created above)
pattern = "*_segments.csv"
results_files = glob(os.path.join(DATA_PATH, pattern))
pprint([(i, fi) for i, fi in enumerate(results_files)])

# Choose a file

In [ ]:
i = 1
result_file = results_files[i]
print(">>> Analyzing segments from {}".format(os.path.split(result_file)[1]))

In [ ]:
# Load results
header, results = load_foof_results(os.path.join(DATA_PATH, result_file))

# Load raw data
data_file = (result_file.split('_')[:-1])
data_file = "_".join(data_file)
data_file += ".mat"
print(">>> Loading {}".format(data_file))

fi = h5py.File(data_file)
X = fi['data'].value
print("Data shape {}".format(X.shape))

fs = float(fi['Fs'].value)
n_samples = float(fi['nSamples'].value) 
T = n_samples / fs
print("Experiment time T {}".format(T))

# Choose a segment

...A row from results

In [ ]:
n = 54

In [ ]:
m, c, i, j, peak, power, bw = results[n]
m = int(m)
c = int(c)
i = int(i)
j = int(j)
peak = float(peak)
power = float(power)
bw = float(bw)

times = create_times(10, 1/fs)
print(times.shape)
print(m, c, i, j, peak, power, bw)

In [ ]:
x = X[i:j, c]

In [ ]:
print(">>> x shape {}".format(x.shape))

# Plot the segment

In [ ]:
# -
x_alpha = butter_bandpass_filter(x, peak - bw, peak + bw, fs, order=2)
x_hil = np.abs(hilbert(x_alpha))
x_filt = butter_bandpass_filter(x, 3, 100, fs, order=2)

p = figure(plot_width=800, plot_height=300)
p.line(times, x_filt, color="black", alpha=0.6)
p.line(times, x_alpha, color="red", alpha=0.6)
p.line(times, x_hil, color="red", line_width=3, alpha=.3)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'LFP (uV)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# -
freqs, psd = create_psd(x, fs, fs)
peak_line = np.linspace(psd.min(), psd.max(), 10)

p = figure(plot_width=300, plot_height=300, x_axis_type="log", y_axis_type="log")
p.line(freqs, psd, color="black", alpha=1)
p.line(x=np.repeat(peak, peak_line.size), y=peak_line)
p.xaxis.axis_label = 'Freq (Hz)'
p.yaxis.axis_label = 'Log power (AU)'
p.x_range = Range1d(1, 50)
# p.y_range = Range1d(-3, 5)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

show(p)